<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision as vision
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/emanhamed/Houses-dataset

fatal: destination path 'Houses-dataset' already exists and is not an empty directory.


In [3]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

Remove all entries with zipcodes occuring less than 20 times

In [4]:
def cleanData(df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df

In [5]:
dataset=cleanData(df)

In [6]:
"""
import shutil
from_prefix="Houses-dataset/Houses Dataset/"
to_prefix="images/"
suffix="_frontal.jpg"
!mkdir -p images
for newidx,oldidx in enumerate(df.index.tolist()):
  oldname=from_prefix+str(oldidx)+suffix
  newname=to_prefix+str(newidx)+suffix
  #print("moving from %s to %s"%(oldname,newname))
  shutil.copy(oldname,newname)
  """

'\nimport shutil\nfrom_prefix="Houses-dataset/Houses Dataset/"\nto_prefix="images/"\nsuffix="_frontal.jpg"\n!mkdir -p images\nfor newidx,oldidx in enumerate(df.index.tolist()):\n  oldname=from_prefix+str(oldidx)+suffix\n  newname=to_prefix+str(newidx)+suffix\n  #print("moving from %s to %s"%(oldname,newname))\n  shutil.copy(oldname,newname)\n  '

In [7]:
#randomize the dataframe
ran_dataset=dataset.sample(len(dataset))

In [8]:
# the dataset has 384 entries. Choose 310 for training and 74 for testing
train_dataset=ran_dataset[0:310]
test_dataset=ran_dataset[310:len(dataset)]
train_dataset.to_csv("train.csv",index_label="index")
test_dataset.to_csv("test.csv",index_label="index")

In [9]:
from torchvision.io import read_image
import os
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    self.imgDir=imgDir
    dataset=pd.read_csv(csvFile)
    dummy=pd.get_dummies(dataset['zipcode'])
    price=dataset['price']
    size=dataset['size']
    self.max_size=size.max()
    self.max_price=price.max()
    size=size/self.max_size
    price=price/self.max_price
    df=dataset.drop(['size','price','zipcode'],axis=1)
    self.data=pd.concat([df,size,dummy,price],axis=1)
    self.resize=vision.transforms.Resize((48,48))
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    #
    img_idx=self.data.iloc[idx,0]
    # the images were labelled starting at 1. Pandas starts at 0
    path=os.path.join(self.imgDir,str(img_idx+1)+"_frontal.jpg")
    img=read_image(path)
    img=self.resize(img)
    return self.data.iloc[idx,1:-1].to_numpy(dtype=np.float32),img,np.float32(self.data.iloc[idx,-1])

In [10]:
train_dataset=CustomDataset("train.csv","Houses-dataset/Houses Dataset/")
test_dataset=CustomDataset("test.csv","Houses-dataset/Houses Dataset/")

In [11]:
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=1,shuffle=False)

In [12]:
itr=iter(train_loader)
data=next(itr)

In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.relu=nn.ReLU() 
    self.fc1=nn.Linear(in_features=11,out_features=32)
    self.fc2=nn.Linear(in_features=32,out_features=16)
    self.fc3=nn.Linear(in_features=16,out_features=1)
  def forward(self,x):
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    x=self.relu(x)
    x=self.fc3(x)
    return x

In [37]:
from torch.optim import SGD,Adam
from torch.nn import MSELoss,L1Loss

model=Net()
#optimizer=SGD(model.parameters(),lr=0.001)
optimizer=Adam(model.parameters())
#loss_fn=MSELoss()
loss_fn=L1Loss()
epochs=50
for epoch in range(epochs):
  for input,img,price in train_loader:
    output=model(input)
    loss=loss_fn(output.squeeze(),price)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(loss)
  



tensor(0.0531, grad_fn=<L1LossBackward0>)
tensor(0.1840, grad_fn=<L1LossBackward0>)
tensor(0.0283, grad_fn=<L1LossBackward0>)
tensor(0.0097, grad_fn=<L1LossBackward0>)
tensor(0.0224, grad_fn=<L1LossBackward0>)
tensor(0.0217, grad_fn=<L1LossBackward0>)
tensor(0.0140, grad_fn=<L1LossBackward0>)
tensor(0.0272, grad_fn=<L1LossBackward0>)
tensor(0.0179, grad_fn=<L1LossBackward0>)
tensor(0.0826, grad_fn=<L1LossBackward0>)
tensor(0.0084, grad_fn=<L1LossBackward0>)
tensor(0.0443, grad_fn=<L1LossBackward0>)
tensor(0.0158, grad_fn=<L1LossBackward0>)
tensor(0.0241, grad_fn=<L1LossBackward0>)
tensor(0.0201, grad_fn=<L1LossBackward0>)
tensor(0.0051, grad_fn=<L1LossBackward0>)
tensor(0.0162, grad_fn=<L1LossBackward0>)
tensor(0.0170, grad_fn=<L1LossBackward0>)
tensor(0.0167, grad_fn=<L1LossBackward0>)
tensor(0.0275, grad_fn=<L1LossBackward0>)
tensor(0.0122, grad_fn=<L1LossBackward0>)
tensor(0.0206, grad_fn=<L1LossBackward0>)
tensor(0.0260, grad_fn=<L1LossBackward0>)
tensor(0.0162, grad_fn=<L1LossBack

In [15]:
itr=iter(test_loader)
max_test=test_dataset.max_price
max_train=train_dataset.max_price
print(max_test,max_train)


2200000 5858000


In [16]:
input,img,price=next(itr)
print(price.item()*max_test)
output=model(input)
output.item()*max_train

628999.9902248383


554011.4190131426

In [38]:
total=0.0
total2=0.0
count=0
max_test=test_dataset.max_price
max_train=train_dataset.max_price
for _,(input,img,price) in enumerate(test_loader):
  count+=1
  output=model(input)
  #print(torch.abs(price*max_test-output.item()*max_train)/(price*max_test))
  abs=(torch.abs(output.item()*max_train-price*max_test)/(price*max_test)).squeeze()
  total+=abs

In [39]:
print(total/count)

tensor(0.2822)
